In [1]:
import os
import requests
import zipfile
import io
import pandas as pd
import geopandas as gpd
import osmnx as ox
import pygris
import matplotlib.pyplot as plt
from shapely.geometry import Point

# ===============================
# 0. SETUP & VERIFICATION
# ===============================
# Verify that all libraries loaded correctly
print(f"Pandas Version: {pd.__version__}")
print(f"GeoPandas Version: {gpd.__version__}")
print(f"OSMnx Version: {ox.__version__}")

Pandas Version: 2.3.3
GeoPandas Version: 1.1.1
OSMnx Version: 2.0.7


In [2]:
# ====================================
# 1. CENTRAL AUSTIN NETWORK ANALYSIS
# ====================================
# Define the place and network type
center_point = (30.2672, -97.7431)
dist_meters = 3000
network_type = "walk"

# Download the street network graph
print(f"\n--- 1. Downloading {network_type} network for Central Austin (3km radius) ---")
G = ox.graph_from_point(
    center_point,
    dist=dist_meters,
    network_type=network_type,
    simplify=True
)
print(f"Graph loaded. Nodes: {len(G.nodes):,}, Edges: {len(G.edges):,}")

# Project the graph for accurate area/distance calcs
print("Projecting graph...")
G_proj = ox.project_graph(G, to_latlong=False)

# Get the total area of the place boundary
import numpy as np
study_area_sq_km = np.pi * (dist_meters / 1000) ** 2

# Calculate Intersection Density
intersection_count = len([n for n, d in G.nodes(data=True) if d.get("street_count", 0) > 1])
intersection_density = intersection_count / study_area_sq_km

print("-" * 50)
print(f"Study Area (3km Radius): {study_area_sq_km:.2f} sq km")
print(f"Total Intersections: {intersection_count:,}")
print(f"Intersection Density: {intersection_density:.2f} / sq km")
print("-" * 50)


--- 1. Downloading walk network for Central Austin (3km radius) ---
Graph loaded. Nodes: 19,221, Edges: 55,112
Projecting graph...
--------------------------------------------------
Study Area (3km Radius): 28.27 sq km
Total Intersections: 14,937
Intersection Density: 528.29 / sq km
--------------------------------------------------


In [3]:
# ======================================
# 2. GRANULAR DATA: CENSUS BLOCK GROUPS
# ======================================
print("\n--- 2. Downloading Census Block Groups for Travis County ---")
bg_travis = pygris.block_groups(state="TX", county="Travis", year=2020, cache=True)

# Filter Block Groups to just those intersecting our study area
study_area_polygon = gpd.GeoDataFrame(
    geometry=[Point(center_point[1], center_point[0])],
    crs="EPSG:4326"
).to_crs(bg_travis.crs).buffer(dist_meters).iloc[0]

# Filter: Keep block groups that intersect our 3 km circle
bg_study_area = bg_travis[bg_travis.intersects(study_area_polygon)].copy()

print(f"Filtered to {len(bg_study_area)} block groups within Central Austin.")


--- 2. Downloading Census Block Groups for Travis County ---
Using FIPS code '48' for input 'TX'
Using FIPS code '453' for input 'Travis'
Filtered to 766 block groups within Central Austin.


/tmp/ipykernel_5634/2236403026.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ).to_crs(bg_travis.crs).buffer(dist_meters).iloc[0]


In [4]:
# ==========================================
# 3. TARGET VARIABLE: EPA WALKABILITY INDEX
# ==========================================
print("\n--- 3. Downloading EPA National Walkability Index Data ---")
epa_url = "https://edg.epa.gov/EPADataCommons/public/OA/WalkabilityIndex.zip"

try:
    print("Fetching ZIP file...")
    r = requests.get(epa_url)
    z = zipfile.ZipFile(io.BytesIO(r.content))

    csv_name = [f for f in z.namelist() if f.endswith('.csv')][0]
    print(f"Reading {csv_name}...")
    
    # Read only necessary columns to save memory
    cols_to_keep =['GEOID10', 'GEOID20', 'NatWalkInd']
    epa_data = pd.read_csv(z.open(csv_name), usecols=lambda c: c in cols_to_keep, dtype={'GEOID10': str, 'GEOID20': str})
    print("EPA Data loaded successfully.")
          
except Exception as e:
    print(f"Error loading EPA data: {e}")


--- 3. Downloading EPA National Walkability Index Data ---
Fetching ZIP file...
Error loading EPA data: list index out of range


In [ ]:
# ======================================
# 4. DATA MERGING
# ======================================
print("\n--- 4. Merging Spatial & Walkability Data ---")
bg_study_area['GEOID'] = bg_study_area['GEOID'].astype(str)
join_col_epa = 'GEOID20' if 'GEOID20' in epa_data.columns else 'GEOID10'

# Merge
austin_walkability = bg_study_area.merge(epa_data, left_on='GEOID', right_on=join_col_epa, how='left')
target_col = 'NatWalkInd'
austin_walkability = austin_walkability.dropna(subset=[target_col])

print(f"Final Dataset Size: {len(austin_walkability)} block groups.")

In [ ]:
# =============================
# 5. VISUALIZATION
# =============================
print("\n --- 5. Visualizing the Target Variable ---")
fig, ax = plt.subplots(figsize=(10, 10))
austin_walkability.plot(column=target_col, cmap='RdYlGn', legend=True, ax=ax)

# Add the study area circle for context
from shapely.geometry import Point
circle = gpd.GeoSeries([Point(center_point[1], center_point[0])],
                        crs="EPSG:4326").to_crs(bg_travis.crs).buffer(dist_meters)
circle.plot(ax=ax, facecolor='non', edgecolor='blue', linewidth=2, linestyle='--')

ax.set_title("EPA National Walkability Index\nCentral Austin (3 km Radius)")
ax.axis('off')
plt.show()